# *Entregable 2 Deep Learning: Redes Convolucionales*
- Rubén Gil Martínez

# **P2.1**

**E1:** Carga las imágenes de entrada y sus identificadores de clase correspondientes. Almacénalos en dos tensores con las dimensiones adecuadas para servir de entrada y salida (entrenamiento) respectivamente de la red neuronal construida.

**E2:** Aleatoriza la posición de las imágenes y consecuentemente los identificadores.

**E3:** Particiona los tensores de entrada en dos, asigna el 80% de las imágenes para el entrenamiento y el 20% para la validación.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import os
from keras.preprocessing.image import load_img, img_to_array

In [5]:
# Ruta del conjunto de datos
data_path = '../data/imgs/SNR_high'

# Dimensiones de las imágenes
img_size = (50, 50)

# Cargar imágenes y etiquetas
images = []
labels = []

for filename in os.listdir(data_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # Extraer la etiqueta del nombre del archivo
        label = str(filename.split('_')[0])  
        
        # Cargar imagen en escala de grises
        img_path = os.path.join(data_path, filename)
        image = load_img(img_path, color_mode='grayscale', target_size=img_size)
        image = img_to_array(image) / 255.0  # Normalizar los valores de los píxeles
        
        images.append(image)
        labels.append(label)

# Convertir listas a arrays de NumPy
images = np.array(images)  # Dimensión (N, 50, 50, 1)
labels = np.array(labels)  # Dimensión (N,)

# Mezclar aleatoriamente las imágenes y etiquetas
indices = np.random.permutation(len(images))
images = images[indices]
labels = labels[indices]

# Partición en entrenamiento (80%) y validación (20%)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Imprimir dimensiones resultantes
print("Tamaño del conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape, y_val.shape)


Tamaño del conjunto de entrenamiento: (400, 50, 50, 1) (400,)
Tamaño del conjunto de validación: (100, 50, 50, 1) (100,)


**E4:** Construye un MLP que realice la clasificación de las imágenes según se especifica en el apartado [MLP baseline](mlp). Utiliza el algoritmo de optimización y la función de pérdidas que creas más oportuna.

**E5:** ¿Cuántos parámetros a entrenar tendría esta red?

**E6:** Entrena el modelo y encuentra la combinación de tamaño de batch y épocas que genere mejores resultados.

**E7:** Muestra la evolución de la función de pérdidas para los conjuntos de datos de entrenamiento y validación a lo largo del proceso de entrenamiento.

**E8:** ¿Qué precisión tiene el modelo?

**E9:** Construye una red neuronal para clasificar las imágenes del dataset que contenga capas convolucionales y un MLP. Utiliza el algoritmo de optimización y la función de pérdidas que creas más oportuna.

**E10:** ¿Cuántos parámetros a entrenar tendría esta red? Compáralos con los que se obtuvieron con el MLP.

**E11:** Entrena el modelo y determina la precisión de la clasificación. Discute los resultados en comparación con los que se obtuvieron con el MLP

**E12:** Muestra la matriz de confusión resultado de procesar el conjunto de datos de test.

**E13:** ¿Qué dos clases presentan mayor confusión mutua? ¿Por qué? Visualiza unas cuantas imágenes con clasificaciones correctas e incorrectas.